# ZarrManager example

In [ ]:
# --- Imports ---
import pymif.microscope_manager as mm
import napari
import os
import time

In [ ]:
# --- Set input data path ---
main_path = "../../pymif_test_data/zarr"
dataset_name = "5T_2C_81Z_2304Y_2304X"
dataset_name = "5T_2C_8Z_128Y_128X"
input_zarr_path = f"{main_path}/{dataset_name}.zarr"


In [ ]:
# --- Initialize manager ---
# Here, the argument chunks can be used to specify a preferred chunk (Default: None = zarr native chunks)
zarr_dataset = mm.ZarrManager(path=input_zarr_path)

In [ ]:
# --- Show metadata summary ---
for i in zarr_dataset.metadata:
    print(f"{i.upper()}: {zarr_dataset.metadata[i]}")

In [ ]:
zarr_dataset.data

## Adding labels

In [ ]:
# import dask.array as da
# from skimage.measure import label
# import numpy as np

# # Function that runs labeling on numpy blocks
# def label_block(block):
#     return label(block > 1000)

# # Use map_blocks for each channel
# label_images = []
# for channel in [0, 1]:

#     label_levels = []
    
#     for t in range(zarr_dataset.data[0].shape[0]):
        
#         slice_t = zarr_dataset.data[0][t, channel]
        
#         labeled = da.map_overlap(
#             label_block,
#             slice_t,
#             depth={0: 1, 1: 10, 2: 10},  # no overlap in Z, 10 px in Y and X
#             boundary='reflect',
#             trim=True,
#             dtype=np.int32,
#         )
        
#         # Add back T-C axis
#         labeled = labeled[None, None, ...]
#         label_levels.append(labeled)
    
#     # Stack back to full (t, z, y, x)
#     label_stack = da.concatenate(label_levels, axis=0)

#     label_images.append([label_stack])

# label_names = ["nuclei", "membranes"]

In [ ]:
# # Create pyramid compliant with raw data dimensions
# from pymif.microscope_manager.utils import pyramid
# label_images = [
#     pyramid.build_pyramid(label_images[0],
#                          zarr_dataset.metadata,
#                          num_levels = 5)[0],
#     pyramid.build_pyramid(label_images[1],
#                          zarr_dataset.metadata,
#                          num_levels = 5)[0]
# ]

# # Remove channel dimension from labels
# label_images = [
#     [l[:,0] for l in label_images[0]],
#     [l[:,0] for l in label_images[1]],
# ]

In [ ]:
# # Add labels to the dataset
# zarr_dataset.add_label(label_images[0], label_names[0])
# zarr_dataset.add_label(label_images[1], label_names[1])

In [ ]:
# # Reload dataset
# zarr_dataset = mm.ZarrManager(path=input_zarr_path, chunks=(1,1,64,1024,1024))

In [ ]:
zarr_dataset.data

In [ ]:
zarr_dataset.labels

In [ ]:
zarr_dataset.metadata

In [ ]:
v = zarr_dataset.visualize(stop_level=2, in_memory=True)